<a href="https://colab.research.google.com/github/20240609-lab/Homework_2025/blob/main/199chatclientGUI2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tkinter import *
import tkinter.ttk as ttk
import threading
import socket

HOST = 'localhost'         # 채팅서버 주소
PORT = 9010                # 채팅서버 포트번호

#lock = threading.Lock()

# 채팅 클라이언트 메인 클래스
class ChatClient():
    def __init__(self):
        self.me = ''        # 채팅클라이언트의 주인의 대화명
        self.title = '파이썬200-채팅클라이언트'
        #self.host = HOST    # 채팅서버 IP 또는 도메인주소
        #self.port = PORT    # 채팅서버 포트번호

        # 채팅 클라이언트 UI를 정의
        root = Tk()
        # 메신저 닫기 했을 때 발생하는 이벤트처리
        root.protocol("WM_DELETE_WINDOW", self.destroyWin)
        self.root = root

        self.root.title(self.title)
        self.root.resizable(width=False, height=False)

        # 화면 아래에 표시될 상태표시창에 출력될 변수
        self.statusmsg = StringVar()

        # 내 대화 메시지
        self.mymsg = StringVar()

        # 메인프레임 정의
        content = ttk.Frame(self.root, padding=(6, 6, 6, 6))
        content.grid(column=0, row=0, sticky=("nwes"))

        panel1 = ttk.Frame(content, relief='groove', padding=(3,3,3,3))
        panel2 = ttk.Frame(content, relief='groove', padding=(3,3,3,3))
        panel3 = ttk.Frame(content, padding=(3,3,3,3))

        panel1.grid(column=0, row=0, sticky=('nwes'))
        panel2.grid(column=0, row=1, sticky=('nwes'))
        panel3.grid(column=0, row=2, sticky=('nwes'))

        # 대화 표시창 생성
        self.textoutwin = Text(panel1, relief='solid', width=65, height=20, font=('맑은 고딕', 9))
        self.textoutysb = ttk.Scrollbar(panel1, orient=VERTICAL, command=self.textoutwin.yview)
        self.textoutwin['yscroll'] = self.textoutysb.set

        # 대화 입력창 생성
        self.textinwin = Entry(panel2, width=65, textvariable=self.mymsg)

        # 상태 표시창 생성
        self.status = ttk.Label(panel3, width=65, textvariable=self.statusmsg)

        # 엔터키와 메시지 전송 함수 바인딩
        self.textinwin.bind('<Return>', self.sendMessage)

        # 대화 표시창 및 스크롤바 위치 시키기
        self.textoutwin.grid(column=0, rowspan=3, row=0, sticky='w')
        self.textoutysb.grid(column=1, rowspan=3, row=0, sticky='ns')
        self.textoutwin.tag_configure('mytalk', foreground='blue', justify='right')
        self.textoutwin.tag_configure('centermsg', foreground='green', justify='center')

        # 대화 입력창 위치 시키기
        self.textinwin.grid(column=0, row=0, sticky='w')

        # 상태 표시창 위치 시키기
        self.status.grid(column=0, row=0, sticky='w')

        # 포커스를 메시지 입력창에 두기
        self.textinwin.focus()

    def connect(self):  # 서버에 연결
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        #self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_KEEPALIVE, 1)

        try:
            self.sock.connect((HOST, PORT))
            t = threading.Thread(target=self.recvMessage)
            t.daemon = True
            t.start()
        except Exception as e:
            self.statusmsg.set(str(e))
            return
        self.statusmsg.set('채팅서버에 연결되었습니다')

    # 엔터키를 누르면 메시지 전송하는 로직
    def sendMessage(self, *args):
        try:
            msg = self.mymsg.get()
            if not msg:
                return

            #with lock:
            self.sock.send(msg.encode())
            self.textinwin.delete(0, END)

            if msg == '/quit':
                self.statusmsg.set('대화종료: 프로그램을 종료하세요')

        except Exception as e:
            self.statusmsg.set(str(e))

    # 메시지 수신 처리를 위한 쓰레드 함수
    def recvMessage(self):
        isLogin = False
        while True:
            try:
                # 대화내용이 화면높이를 넘어설때 자동 스크롤
                self.textoutwin.see(END)

                msg = self.sock.recv(65565)
                if not msg:
                    break

                msg = msg.decode()

                if '/welcome' in msg:
                    username = msg.split(';')[1]
                    msg = msg.split(';')[2]

                    if not isLogin:
                        self.me = username
                        isLogin = True

                    self.textoutwin.insert(END, username+msg+'\n', 'centermsg')
                    continue

                if '/login' in msg:
                    msg = msg.split(';')[1]
                    self.textoutwin.insert(END, msg+'\n')
                    continue

                if msg[0] == '#':
                    username = msg.split('#')[1]
                    msg = msg.split('#')[2]
                    if self.me == username:
                        self.textoutwin.insert(END, msg+'\n', 'mytalk')
                    else:
                        msg = f'[{username}]\n\t{msg}'
                        self.textoutwin.insert(END, msg+'\n')
                else:
                    self.textoutwin.insert(END, msg+'\n', 'centermsg')

            except Exception as e:
                pass

    # 메신저 실행 함수
    def run(self):
        self.connect()
        self.root.mainloop()
        self.sock.close()

    # 메신저 종료시 호출되는 함수
    def bye(self):
        try:
            self.sock.send('/quit'.encode())
        except:
            pass

    # 메신저 윈도우 디스트로이
    def destroyWin(self):
        self.bye()
        self.root.destroy()

if __name__ == '__main__':
    chatgui = ChatClient()
    try:
        chatgui.run()
    except:
        pass